In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### load data

In [ ]:
sale_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv', parse_dates=['date'])
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')


In [ ]:
def EDA(df):
    print("-------df.head(5)-------")
    print(df.head(5))
    print("-------INFO----------")
    print(df.info())
    print("-------describe--------")
    print(df.describe())
    print("-------data type-------")
    print(df.dtypes)
    print("-------missing num-------")
    print(df.isna().sum())

In [ ]:
EDA(sale_data)

In [ ]:
dataset = sale_data.pivot_table(index=['shop_id', 'item_id'], values=['item_cnt_day'], columns=['date_block_num'],
                                fill_value=0, aggfunc='sum')

In [ ]:
dataset

In [ ]:
dataset.reset_index(inplace=True)

In [ ]:
dataset

In [ ]:
dataset = pd.merge(test, dataset, on=['item_id', 'shop_id'], how='left')

In [ ]:
dataset

In [ ]:
dataset.fillna(0, inplace=True)

In [ ]:
dataset

In [ ]:
dataset.drop(['shop_id', 'item_id', 'ID'], inplace=True, axis=1)
dataset


In [ ]:
x_train = np.expand_dims(dataset.values[:, :-1], axis=2)
y = dataset.values[:, -1:]

x_test = np.expand_dims(dataset.values[:, 1:], axis=2)

print(x_train.shape, y.shape, x_test.shape)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(x_train.shape[1], x_train.shape[2]),
                         return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='relu')
])


In [ ]:
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
history = model.fit(x_train, y, epochs=10, batch_size=256)


In [ ]:
predictions = model.predict(x_test)

In [ ]:
import csv

with open('sample_submission.csv', 'w')as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'item_cnt_month'])
    for i in range(214200 - 1):
        writer.writerow([i, predictions[i][0]])

In [ ]:
predictions